#**Implementing a mental well-being assistant**

#STEP 1: Setting up the Environment

#Install the relavent libraries

* Transformers: A Python library that provides pre-trained NLP models for various tasks.

*  FAISS: Enables efficient similarity search and clustering of high-dimensional vectors.

* Sentence Transformers: Generates high-quality sentence embeddings using transformer models, facilitating tasks that require understanding sentence-level semantics.




In [ ]:
!pip install transformers faiss-gpu sentence-transformers

Importing the libraries

In [ ]:
import pandas as pd  # Import Pandas library for data handling and analysis
from sentence_transformers import SentenceTransformer  # Import SentenceTransformer for sentence embeddings
import faiss  # Import FAISS for efficient similarity search and clustering
from transformers import pipeline  # Import pipeline from Transformers for NLP tasks

#Step2: Generating responses Using Hugging Face Transformers Pipeline **without context**

#1. Function for generating response for a given Querry and a specific model

In [ ]:

def generate_response(query, model_name):

    # Initialize the pipeline with the specified model
    generator = pipeline('text-generation', model=model_name)

    # Formulate the prompt with the context and query
    prompt = f"Answer the question\nQuestion: {query}\nAnswer: "

    # Generate response using the pipeline
    response = generator(prompt, max_new_tokens=50, num_return_sequences=1)
    return response[0]['generated_text']


##Model 1: gpt2

In [ ]:
query = input("Ask a question about mental well-being") #take input
response = generate_response(query, 'gpt2')  #generate response using the gpt2 model
print(response)

Ask a question about mental well-beinghow to reduce the stress


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer the question
Question: how to reduce the stress
Answer:  Well, there are two ways - I can reduce stress simply by increasing the number of posts. If you want to avoid some of the high stress posts with your message, you can follow the guidelines above and remove the  posts mentioned below for


##Step3: Generating responses using **RAG**

Retrieval-Augmented Generation (RAG) is a technique that combines retrieval-based methods with generative models to enhance the quality of generated content. It retrieves relevant information from a knowledge base or dataset and uses this context to generate more accurate and contextually appropriate responses.

##1. Loading the Dataset to Serve as a Knowledge Base

In [ ]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

# Path to your dataset in Google Drive
file_path = '/content/drive/MyDrive/UOP/NLP/qna.csv' #Change the file path to where the Question&Answer csv file is located

# Load the dataset
df = pd.read_csv(file_path)
questions = df['question'].tolist()
answers = df['answer'].tolist()

# Display the first few rows to ensure the data is loaded correctly
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,question,answer
0,What are some effective ways to reduce stress?,Effective ways to reduce stress include regula...
1,How can I improve my sleep quality?,"To improve sleep quality, establish a regular ..."
2,What should I do if I feel anxious all the time?,"If you feel anxious all the time, consider tal..."
3,How can I boost my mood naturally?,Boosting your mood naturally can be achieved t...
4,What are some signs of burnout?,"Signs of burnout include feeling exhausted, de..."


##2. Embedding the questions list *using* sentence Transformers

Sentence embeddings convert sentences into dense numerical vectors that capture their semantic information (meaning).

In [ ]:
# Load the SentenceTransformer model 'all-mpnet-base-v2'
model = SentenceTransformer('all-mpnet-base-v2')

# Generate embeddings for the list of questions using the loaded model
question_embeddings = model.encode(questions)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

##3. Initializing and Populating FAISS Index with Question Embeddings
FAISS is used to create an index of sentence embeddings for efficient similarity searches. This allows quick retrieval of relevant embeddings. These retrieved embeddings can then improve the accuracy of generative model responses.

In [ ]:
# Initialize the FAISS index
dimension = question_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
index.add(question_embeddings)

##4. Function for Retrieve Top k Similar Questions and Answers Using FAISS Index


In [ ]:
def retrieve(query, k=5):
    # Encode the query using the loaded SentenceTransformer model
    query_embedding = model.encode([query])

    # Use the FAISS index to search for the nearest neighbors (similar embeddings)
    distances, indices = index.search(query_embedding, k)

    # Return the top k closest questions and their corresponding answers
    return [questions[i] for i in indices[0]], [answers[i] for i in indices[0]]


##5. Creating a interactive chat bot
This time, the **generate_response** function uses the answers retrieved from the knowledge base in the prompt to get a more accurate response from the model.

**chat_bot** function retrives the questions and answers relavant to the user's query and pass it on to **generate_response** function.

In [ ]:

# Generating Response with the content retrived from the csv file
def generate_response(retrieved_questions, retrieved_answers, query, model_name):

    generator = pipeline('text-generation', model=model_name)
    # Combine retrieved answers into a context
    context = " ".join(retrieved_answers)
    # Formulate the prompt with the context and query
    prompt = f"Answer the question using the context below.\nContext: {context}\nQuestion: {query}\nAnswer: "
    response = generator(prompt, max_new_tokens=50, num_return_sequences=1)
    return response[0]['generated_text']


def chat_bot(query, model):
    # Retrieve top questions and answers related to the input query
    retrieved_questions, retrieved_answers = retrieve(query)
    # Generate a response based on the retrieved questions and answers
    response = generate_response(retrieved_questions, retrieved_answers, query, model)
    # Return the generated response
    return response



##Model1: GPT-2

In [ ]:
query = input("Ask a question about mental well-being")
response = chat_bot(query, 'gpt2')
print(response)

Ask a question about mental well-beinghow to reduce stress


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer the question using the context below.
Context: Effective ways to reduce stress include regular physical activity, practicing mindfulness or meditation, getting enough sleep, staying connected with loved ones, and managing time effectively. Effective ways to cope with stress include regular physical activity, practicing mindfulness or meditation, maintaining a healthy diet, and seeking support from friends and family. Effective time management involves prioritizing tasks, breaking large projects into smaller steps, setting deadlines, avoiding multitasking, and taking regular breaks. Planning your day and sticking to a schedule can also help. Effective ways to reduce anxiety include practicing mindfulness, engaging in physical activity, maintaining a healthy diet, and seeking professional help if needed. Relaxation techniques include deep breathing exercises, progressive muscle relaxation, guided imagery, mindfulness meditation, and yoga.
Question: how to reduce stress
Answer:    

#STEP3: Trying out Different models

Text generation model is defined in the ***generate_response*** function by line

generator = pipeline('text-generation', model=model_name)

More models from Hugging Face's transformer library

*https://huggingface.co/models?pipeline_tag=text-generation&library=transformers&sort=trending*


##Model2: DistilGPT-2 is a smaller, faster, and lighter version of the GPT-2 model

In [ ]:
 query = input("Ask a question about mental well-being: ")
response = chat_bot(query, 'distilgpt2')
print(response)

Ask a question about mental well-being: how to reduce stress


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer the question using the context below.
Context: Effective ways to reduce stress include regular physical activity, practicing mindfulness or meditation, getting enough sleep, staying connected with loved ones, and managing time effectively. Effective ways to cope with stress include regular physical activity, practicing mindfulness or meditation, maintaining a healthy diet, and seeking support from friends and family. Effective time management involves prioritizing tasks, breaking large projects into smaller steps, setting deadlines, avoiding multitasking, and taking regular breaks. Planning your day and sticking to a schedule can also help. Effective ways to reduce anxiety include practicing mindfulness, engaging in physical activity, maintaining a healthy diet, and seeking professional help if needed. Relaxation techniques include deep breathing exercises, progressive muscle relaxation, guided imagery, mindfulness meditation, and yoga.
Question: how to reduce stress
Answer:  
A

##Model3: GPT-2 Medium is a mid-sized version of the GPT-2 model, offering a balance between performance and computational efficiency.

In [ ]:
query = input("Ask a question about mental well-being: ")
response = chat_bot(query, 'gpt2-medium')
print(response)

Ask a question about mental well-being: how to improve mental health


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer the question using the context below.
Context: Promoting mental well-being involves practicing self-care, staying connected with loved ones, engaging in meaningful activities, and seeking professional help if needed. Mindfulness can help reduce stress, improve focus, enhance emotional regulation, and promote a sense of calm and well-being. Seeking professional help can provide guidance, support, and effective strategies for managing mental health challenges. Physical activity can boost mood, reduce symptoms of anxiety and depression, improve sleep, and increase overall energy levels. Supporting mental health during a pandemic involves staying connected with loved ones, maintaining a routine, practicing self-care, and seeking professional help if needed.
Question: how to improve mental health
Answer:  Panda (2011) and this article give some ideas on how to support the quality of a person's life using mindfulness. However, the best way is to encourage a sense of well-being and sup

### Step 4: Changing the Parameters

```python
response = generator(prompt, max_new_tokens=100, num_return_sequences=1)
```

You can modify the generator parameters to see different effects:

- **`max_new_tokens`**: Increasing this value will produce longer responses.
- **`num_return_sequences`**: Increasing this value will generate a variety of different responses.

In [ ]:
#Change the parameters and run it again